In [1]:
import numpy as np
import cv2


import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler


import torchvision
from torchvision import transforms
from torchvision import datasets
from torchvision import transforms

from torchmetrics import StructuralSimilarityIndexMeasure


# Loss Function: SSIM
# Activation Function: ReLu

# TODO:
# input: 640 x 480
# output: 1920 x 1440

In [2]:
BATCH_SIZE = 100
LEARNING_RATE = 0.01
MOMENTUM = 0.9
NUM_WORKERS = 8

INPUT_DIRECTORY = 'Data-Generator/Set1/training_data/'
TRUTH_DIRECTORY = 'Data-Generator/Set1/ground_truth/'

VAL_DIRECTORY = 'Data-Generator/Set2/training_data/'
VAL_TRUTH_DIRECTORY = 'Data-Generator/Set2/ground_truth/'

INPUT_TENSOR_DIR = 'tensors/Set1/training_data/'
TRUTH_TENSOR_DIR = 'tensors/Set1/ground_truth/'

VAL_TENSOR_DIR = 'tensors/Set2/training_data/'
VAL_TRUTH_TENSOR_DIR = 'tensors/Set2/ground_truth/'

# Data Generation

In [17]:
def get_data_paths(input_dir, ground_dir, num_frames):
    data_paths = []
    for i in range (1, num_frames - 1):
        data_paths.append([
            input_dir + str(i-1) + '.jpg',
            input_dir + str(i) + '.jpg',
            input_dir + str(i+1) + '.jpg',
            ground_dir + str(i) + '.jpg'
        ])
    return data_paths

def get_data_paths_offset(input_dir, ground_dir, num_frames, offset):
    data_paths = []
    for i in range (1, num_frames - 1):
        data_paths.append([
            input_dir + str(offset + i - 1) + '.jpg',
            input_dir + str(offset + i    ) + '.jpg',
            input_dir + str(offset + i + 1) + '.jpg',
            ground_dir + str(offset + i   ) + '.jpg'
        ])
    print(len(data_paths))
    return data_paths

In [7]:
def get_tensor(path):
    image_rgb = cv2.imread(path)
    image_YCrCb = cv2.cvtColor(image_rgb, cv2.COLOR_BGR2YCrCb)
    image_tensor = torch.tensor(image_YCrCb, dtype=torch.float32)
    image_tensor = image_tensor / 255.0
    return image_tensor

In [23]:
def generate_dataset(data_dir, truth_dir, data_paths):
    for idx in range(len(data_paths)):
        tensors = []
        for i in range(4):
            tensors.append(get_tensor(data_paths[idx][i]))
            
        stacked_frames_tensor = torch.cat([tensors[0], tensors[1], tensors[2]], dim=0)
        ground_truth_tensor = tensors[3]
            
        # Define the path where you want to save the tensor
        save_path_training = data_dir + str(idx) + '.pt'
        save_path_truth = truth_dir + str(idx) + '.pt'

        torch.save(stacked_frames_tensor, save_path_training)
        torch.save(ground_truth_tensor, save_path_truth)

In [27]:
class VHSDataSet(Dataset):
    def __init__(self, data_paths, transform=None, input_tensor_dir='temp1/', truth_tensor_dir='temp2/'):
        self.data_paths = data_paths
        self.transform = transform
        self.input_tensor_dir = input_tensor_dir
        self.truth_tensor_dir = truth_tensor_dir

    def __len__(self):
        return len(self.data_paths)

    def __getitem__(self, idx):
        stacked_frames_tensor = torch.load(input_tensor_dir + str(idx) + '.pt')
        ground_truth_tensor = torch.load(truth_tensor_dir + str(idx) + '.pt')
        if idx % 10 == 0:
            print(idx)
            print(stack_frames_tensor)
            print(ground_truth_tensor + '\n')
#         tensors = []
#         for i in range(4):
#             tensors.append(get_tensor(self.data_paths[idx][i]))

#         stacked_frames_tensor = torch.cat([tensors[0], tensors[1], tensors[2]], dim=0)
#         ground_truth_tensor = tensors[3]
        return stacked_frames_tensor, ground_truth_tensor

In [ ]:
# Create a DataLoader instance for training

# data_paths = get_data_paths(INPUT_DIRECTORY, TRUTH_DIRECTORY, 2400)
# dataset = VHSDataSet(data_paths)
# dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

# Training

In [7]:
# class ResNet(nn.Module):
#     def __init__(self, block, layers, num_classes=1000):
#         super(ResNet, self).__init()
#         self.in_channels = 64
#         self.conv1 = nn.Conv3d(3, 64, kernel_size=8, stride=2, padding=3, bias=False)
#         self.bn1 = nn.BatchNorm2d(64)
#         self.relu = nn.ReLU(inplace=True)
#         self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
#         self.layer1 = self._make_layer(block, 64, layers[0])
#         self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
#         self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
#         self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
#         self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        
#     def _make_layer(self, block, out_channels, blocks, stride=1):
#         layers = []
#         layers.append(block(self.in_channels, out_channels, stride))
#         self.in_channels = out_channels * block.expansion
#         for _ in range(1, blocks):
#             layers.append(block(self.in_channels, out_channels))
#         return nn.Sequential(*layers)

#     def forward(self, x):
#         x = self.conv1(x)
#         x = self.bn1(x)
#         x = self.relu(x)
#         x = self.maxpool(x)

#         x = self.layer1(x)
#         x = self.layer2(x)
#         x = self.layer3(x)
#         x = self.layer4(x)

#         x = self.avgpool(x)
#         x = x.view(x.size(0), -1)
#         return x


In [8]:
# class ResidualBlock(nn.Module):
#     def __init__(self, in_channels, out_channels, stride=1):
#         super(ResidualBlock, self).__init()
#         self.conv1 = nn.Conv3d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
#         self.bn1 = nn.BatchNorm2d(out_channels)
#         self.relu = nn.ReLU(inplace=True)
#         self.conv2 = nn.Conv3d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
#         self.bn2 = nn.BatchNorm2d(out_channels)

#     def forward(self, x):
#         residual = x

#         out = self.conv1(x)
#         out = self.bn1(out)
#         out = self.relu(out)

#         out = self.conv2(out)
#         out = self.bn2(out)

#         out += residual
#         out = self.relu(out)

#         return out


In [12]:
class VHSResNet(nn.Module):
    def __init__(self, block, layers, num_classes=1000):
        super(VHSResNet, self).__init__()
        self.in_channels = 64

        # 3D Convolutional layers for processing stacked frames
        self.conv3d_1 = nn.Conv3d(3, 64, kernel_size=(3, 7, 7), stride=(1, 2, 2), padding=(1, 3, 3), bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Residual layers with 3D convolutions
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=(1, 2, 2))
        self.layer3 = self._make_layer(block, 256, layers[2], stride=(1, 2, 2))
        self.layer4 = self._make_layer(block, 512, layers[3], stride=(1, 2, 2))

        # 2D Transposed Convolutional layers for upscaling
        self.upscale1 = nn.ConvTranspose2d(512 * block.expansion, 256, kernel_size=4, stride=2, padding=1, bias=False)
        self.upscale2 = nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=False)
        self.upscale3 = nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1, bias=False)

        # Output layer
        self.output_layer = nn.ConvTranspose2d(64, 3, kernel_size=7, stride=2, padding=3, bias=False)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        layers = []
        layers.append(block(self.in_channels, out_channels, stride))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        # Input x is a 5D tensor with dimensions (batch_size, temporal_size, channels, height, width)
        batch_size, temporal_size, channels, height, width = x.size()

        # Reshape the input tensor for 3D convolution
        x = x.view(batch_size * temporal_size, channels, height, width)

        # 3D Convolution
        x = self.conv3d_1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        # Residual layers with 3D convolutions
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        # Reshape back to 4D tensor for 2D transpose convolutions
        x = x.view(batch_size, temporal_size, -1, x.size(2), x.size(3))

        # 2D Transposed Convolutions for upscaling
        x = self.upscale1(x)
        x = self.upscale2(x)
        x = self.upscale3(x)

        # Output layer
        x = self.output_layer(x)

        return x

class ResidualBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += residual
        out = self.relu(out)

        return out


In [22]:
train_data_paths = get_data_paths_offset(INPUT_DIRECTORY, TRUTH_DIRECTORY, 1000, 6000)
generate_dataset(INPUT_TENSOR_DIR, TRUTH_TENSOR_DIR, train_data_paths)
train_dataset = VHSDataSet(data_paths=train_data_paths, input_tensor_dir = INPUT_TENSOR_DIR, truth_tensor_dir = TRUTH_TENSOR_DIR)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

998
tensors/Set1/training_data/0.pt

tensors/Set1/training_data/1.pt

tensors/Set1/training_data/2.pt

tensors/Set1/training_data/3.pt

tensors/Set1/training_data/4.pt

tensors/Set1/training_data/5.pt

tensors/Set1/training_data/6.pt

tensors/Set1/training_data/7.pt

tensors/Set1/training_data/8.pt

tensors/Set1/training_data/9.pt

tensors/Set1/training_data/10.pt

tensors/Set1/training_data/11.pt

tensors/Set1/training_data/12.pt

tensors/Set1/training_data/13.pt

tensors/Set1/training_data/14.pt

tensors/Set1/training_data/15.pt

tensors/Set1/training_data/16.pt

tensors/Set1/training_data/17.pt

tensors/Set1/training_data/18.pt

tensors/Set1/training_data/19.pt

tensors/Set1/training_data/20.pt

tensors/Set1/training_data/21.pt

tensors/Set1/training_data/22.pt

tensors/Set1/training_data/23.pt

tensors/Set1/training_data/24.pt

tensors/Set1/training_data/25.pt

tensors/Set1/training_data/26.pt

tensors/Set1/training_data/27.pt

tensors/Set1/training_data/28.pt

tensors/Set1/trainin

tensors/Set1/training_data/240.pt

tensors/Set1/training_data/241.pt

tensors/Set1/training_data/242.pt

tensors/Set1/training_data/243.pt

tensors/Set1/training_data/244.pt

tensors/Set1/training_data/245.pt

tensors/Set1/training_data/246.pt

tensors/Set1/training_data/247.pt

tensors/Set1/training_data/248.pt

tensors/Set1/training_data/249.pt

tensors/Set1/training_data/250.pt

tensors/Set1/training_data/251.pt

tensors/Set1/training_data/252.pt

tensors/Set1/training_data/253.pt

tensors/Set1/training_data/254.pt

tensors/Set1/training_data/255.pt

tensors/Set1/training_data/256.pt

tensors/Set1/training_data/257.pt

tensors/Set1/training_data/258.pt

tensors/Set1/training_data/259.pt

tensors/Set1/training_data/260.pt

tensors/Set1/training_data/261.pt

tensors/Set1/training_data/262.pt

tensors/Set1/training_data/263.pt

tensors/Set1/training_data/264.pt

tensors/Set1/training_data/265.pt

tensors/Set1/training_data/266.pt

tensors/Set1/training_data/267.pt

tensors/Set1/trainin

tensors/Set1/training_data/476.pt

tensors/Set1/training_data/477.pt

tensors/Set1/training_data/478.pt

tensors/Set1/training_data/479.pt

tensors/Set1/training_data/480.pt

tensors/Set1/training_data/481.pt

tensors/Set1/training_data/482.pt

tensors/Set1/training_data/483.pt

tensors/Set1/training_data/484.pt

tensors/Set1/training_data/485.pt

tensors/Set1/training_data/486.pt

tensors/Set1/training_data/487.pt

tensors/Set1/training_data/488.pt

tensors/Set1/training_data/489.pt

tensors/Set1/training_data/490.pt

tensors/Set1/training_data/491.pt

tensors/Set1/training_data/492.pt

tensors/Set1/training_data/493.pt

tensors/Set1/training_data/494.pt

tensors/Set1/training_data/495.pt

tensors/Set1/training_data/496.pt

tensors/Set1/training_data/497.pt

tensors/Set1/training_data/498.pt

tensors/Set1/training_data/499.pt

tensors/Set1/training_data/500.pt

tensors/Set1/training_data/501.pt

tensors/Set1/training_data/502.pt

tensors/Set1/training_data/503.pt

tensors/Set1/trainin

tensors/Set1/training_data/712.pt

tensors/Set1/training_data/713.pt

tensors/Set1/training_data/714.pt

tensors/Set1/training_data/715.pt

tensors/Set1/training_data/716.pt

tensors/Set1/training_data/717.pt

tensors/Set1/training_data/718.pt

tensors/Set1/training_data/719.pt

tensors/Set1/training_data/720.pt

tensors/Set1/training_data/721.pt

tensors/Set1/training_data/722.pt

tensors/Set1/training_data/723.pt

tensors/Set1/training_data/724.pt

tensors/Set1/training_data/725.pt

tensors/Set1/training_data/726.pt

tensors/Set1/training_data/727.pt

tensors/Set1/training_data/728.pt

tensors/Set1/training_data/729.pt

tensors/Set1/training_data/730.pt

tensors/Set1/training_data/731.pt

tensors/Set1/training_data/732.pt

tensors/Set1/training_data/733.pt

tensors/Set1/training_data/734.pt

tensors/Set1/training_data/735.pt

tensors/Set1/training_data/736.pt

tensors/Set1/training_data/737.pt

tensors/Set1/training_data/738.pt

tensors/Set1/training_data/739.pt

tensors/Set1/trainin

tensors/Set1/training_data/948.pt

tensors/Set1/training_data/949.pt

tensors/Set1/training_data/950.pt

tensors/Set1/training_data/951.pt

tensors/Set1/training_data/952.pt

tensors/Set1/training_data/953.pt

tensors/Set1/training_data/954.pt

tensors/Set1/training_data/955.pt

tensors/Set1/training_data/956.pt

tensors/Set1/training_data/957.pt

tensors/Set1/training_data/958.pt

tensors/Set1/training_data/959.pt

tensors/Set1/training_data/960.pt

tensors/Set1/training_data/961.pt

tensors/Set1/training_data/962.pt

tensors/Set1/training_data/963.pt

tensors/Set1/training_data/964.pt

tensors/Set1/training_data/965.pt

tensors/Set1/training_data/966.pt

tensors/Set1/training_data/967.pt

tensors/Set1/training_data/968.pt

tensors/Set1/training_data/969.pt

tensors/Set1/training_data/970.pt

tensors/Set1/training_data/971.pt

tensors/Set1/training_data/972.pt

tensors/Set1/training_data/973.pt

tensors/Set1/training_data/974.pt

tensors/Set1/training_data/975.pt

tensors/Set1/trainin

In [24]:
val_data_paths = get_data_paths_offset(VAL_DIRECTORY, VAL_TRUTH_DIRECTORY, 200, 3800)
generate_dataset(VAL_TENSOR_DIR, VAL_TRUTH_TENSOR_DIR, val_data_paths)
val_dataset = VHSDataSet(data_paths=val_data_paths, input_tensor_dir = VAL_TENSOR_DIR, truth_tensor_dir = VAL_TRUTH_TENSOR_DIR)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

198


In [28]:
model = VHSResNet(block=ResidualBlock, layers=[2, 2, 2, 2])
ssim_loss = StructuralSimilarityIndexMeasure(data_range=1.0)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)

        # Convert outputs and targets to images for SSIM calculation
        outputs_img = to_pil_image(outputs.clamp(0, 1))
        targets_img = to_pil_image(targets.clamp(0, 1))

        # Calculate SSIM loss
        loss = 1 - ssim_loss(outputs_img, targets_img)

        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs)

            # Convert outputs and targets to images for SSIM calculation
            outputs_img = to_pil_image(outputs.clamp(0, 1))
            targets_img = to_pil_image(targets.clamp(0, 1))

            # Calculate SSIM loss
            val_loss = 1 - ssim_loss(outputs_img, targets_img)

    print(f"Epoch {epoch + 1}/{num_epochs}, Training SSIM Loss: {loss.item()}, Validation SSIM Loss: {val_loss.item()}")

# Save the trained model
torch.save(model.state_dict(), "vhs_restoration_model.pth")


PicklingError: Can't pickle <class '__main__.VHSDataSet'>: it's not the same object as __main__.VHSDataSet

In [ ]:
START: 7:04 PM
